In [13]:
import random
import hashlib
from typing import Tuple

In [14]:
PRIME = 23 
GENERATOR = 5  

print(f"\n{'='*80}")
print("PUBLIC PARAMETERS (Known to everyone)")
print(f"{'='*80}")
print(f"Prime (p): {PRIME}")
print(f"Generator (g): {GENERATOR}")


PUBLIC PARAMETERS (Known to everyone)
Prime (p): 23
Generator (g): 5


In [15]:
def generate_private_key(max_value: int = 100) -> int:
    """Generate a random private key"""
    return random.randint(2, max_value)

def generate_public_key(private_key: int, generator: int, prime: int) -> int:
    """Generate public key using: (generator^private_key) mod prime"""
    return pow(generator, private_key, prime)

def generate_shared_secret(other_public_key: int, own_private_key: int, prime: int) -> int:
    """Generate shared secret using: (other_public_key^own_private_key) mod prime"""
    return pow(other_public_key, own_private_key, prime)

In [16]:
print(f"\n{'='*80}")
print("ALICE'S KEY GENERATION")
print(f"{'='*80}")

# Alice generates her private key
alice_private_key = generate_private_key()
print(f"Alice's private key (a): {alice_private_key} [SECRET - kept private]")

# Alice computes her public key: A = g^a mod p
alice_public_key = generate_public_key(alice_private_key, GENERATOR, PRIME)
print(f"Alice's public key (A): {alice_public_key} [PUBLIC - will be sent to Bob]")
print(f"Calculation: A = {GENERATOR}^{alice_private_key} mod {PRIME} = {alice_public_key}")


ALICE'S KEY GENERATION
Alice's private key (a): 51 [SECRET - kept private]
Alice's public key (A): 17 [PUBLIC - will be sent to Bob]
Calculation: A = 5^51 mod 23 = 17


In [17]:
print(f"\n{'='*80}")
print("BOB'S KEY GENERATION")
print(f"{'='*80}")

# Bob generates his private key
bob_private_key = generate_private_key()
print(f"Bob's private key (b): {bob_private_key} [SECRET - kept private]")

# Bob computes his public key: B = g^b mod p
bob_public_key = generate_public_key(bob_private_key, GENERATOR, PRIME)
print(f"Bob's public key (B): {bob_public_key} [PUBLIC - will be sent to Alice]")
print(f"Calculation: B = {GENERATOR}^{bob_private_key} mod {PRIME} = {bob_public_key}")


BOB'S KEY GENERATION
Bob's private key (b): 48 [SECRET - kept private]
Bob's public key (B): 4 [PUBLIC - will be sent to Alice]
Calculation: B = 5^48 mod 23 = 4
